In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm

In [2]:
class Dictionary(object):
    def __init__(self):
        self.word2index = {}
        self.index2word = {}
        self.index = 0

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.index
            self.index2word[self.index] = word
            self.index += 1

    def __len__(self):
        return len(self.word2index)

In [7]:
class TextProcess(object):

    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size=20):
        with open(path, 'r') as file:
            tokens = 0
            for line in file:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)

        # Create a 1-D tensor that contains the index of all the words in the file
        rep_tensor = torch.LongTensor(tokens)
        index = 0
        with open(path, 'r') as file:
            for line in file:
                words = line.split() + ['<eos>']
                for word in words:
                    rep_tensor[index] = self.dictionary.word2index[word]
                    index += 1
        
        num_batches = rep_tensor.shape[0] // batch_size # Find out how many batches are needed
        rep_tensor = rep_tensor[:num_batches * batch_size] # Remove remainder
        rep_tensor = rep_tensor.view(batch_size, -1) # (batch_size, num_batches)

        return rep_tensor

In [8]:
embed_size = 128 # Input features to the LSTM
hidden_size = 1024 # Number of LSTM units
num_layers = 1
num_epochs = 20
batch_size = 20
timesteps = 30
learning_rate = 0.002

In [9]:
corpus = TextProcess()

In [10]:
rep_tensor = corpus.get_data('./data/alice-in-wonderland.txt', batch_size)

In [11]:
vocab_size = len(corpus.dictionary)
print(vocab_size)

5290


In [13]:
num_batches = rep_tensor.shape[1] // timesteps
print(num_batches)

49


In [ ]:
class TextGenerator(nn.Module):

    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(TextGenerator, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        x = self.embed(x) # Perform word embedding
        out, (h, c) = self.lstm(x, h)
        out = out.reshape(out.size(0) * out.size(1), out.size(2))
        out = self.linear(out)
        return out, (h, c)

